# 🚗 BEV Segmentation Test

Load trained model → Run inference → Compare with ground truth → Visualize results


In [18]:
# --- Imports ---
import sys
import os
sys.path.append("..")

import torch
import yaml
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from src.model import BEVSegmentationModel
from src.training import MetricsCalculator
from src.visualization import create_default_visualizer

print("✅ Imports successful")


✅ Imports successful


In [19]:
# --- Load Configuration & Trained Model ---
import glob

# Load config from the specific run
config_path = "../runs/run_20250915_041348/config.yaml"
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

device = torch.device(config['device'])

# Initialize model architecture
model = BEVSegmentationModel(config_path=config_path).to(device)

# Load trained weights
checkpoint_pattern = "../runs/run_20250915_041348/checkpoints/best_model.pth"
if os.path.exists(checkpoint_pattern):
    print(f"📁 Loading trained model from: {checkpoint_pattern}")
    
    # Load checkpoint
    checkpoint = torch.load(checkpoint_pattern, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    print(f"✅ Trained model loaded!")
    print(f"📊 Training epoch: {checkpoint.get('epoch', 'unknown')}")
    
    # Handle metrics safely
    best_miou = checkpoint.get('best_val_miou', None)
    train_loss = checkpoint.get('train_loss', None)
    
    if best_miou is not None:
        print(f"📊 Best validation mIoU: {best_miou:.4f}")
    else:
        print(f"📊 Best validation mIoU: unknown")
        
    if train_loss is not None:
        print(f"📊 Training loss: {train_loss:.4f}")
    else:
        print(f"📊 Training loss: unknown")
    
else:
    print("❌ No trained model found!")
    print("Available checkpoints:")
    checkpoints = glob.glob("../runs/run_20250915_041348/checkpoints/*.pth")
    for ckpt in checkpoints:
        print(f"  - {os.path.basename(ckpt)}")
    raise FileNotFoundError("No trained model checkpoint found!")

print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters()):,}")


Initializing BEV Segmentation Model...
  - Config loaded from: ../runs/run_20250915_041348/config.yaml
  - Backbone: resnet50
  - Pretrained: True
  - Freeze backbone: False
  - Number of classes: 7
  - BEV size: 256x256
Initializing resnet50 backbone...
  - Pretrained: True
  - Freeze backbone: False
  - Feature dimension: 2048 -> 64
  - Output stride: 32
Initializing BEV Encoder...
  - Input channels: 64
  - Hidden channels: 128
  - Output channels: 128
  - Encoder architecture: 64 -> 128 -> 128
  - Downsampling: 256x256 -> 128x128 -> 64x64
Initializing Decoder...
  - Input channels: 128
  - Number of classes: 7
  - Target BEV size: 256x256
  - Decoder architecture: 128 -> 128 -> 64 -> 7
  - Upsampling: 64x64 -> 128x128 -> 256x256
  - Temporal fusion disabled
  - All components initialized successfully
📁 Loading trained model from: ../runs/run_20250915_041348/checkpoints/best_model.pth


C:\Users\Aftab\AppData\Local\Temp\ipykernel_21024\255893444.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_pattern, map_location=dev

✅ Trained model loaded!
📊 Training epoch: 4
📊 Best validation mIoU: unknown
📊 Training loss: unknown
📊 Model parameters: 24,752,071


In [42]:
# --- Load Random Examples from Validation Set ---
import random
import glob

def load_image(path, target_size=(256, 256)):
    """Load and preprocess image"""
    img = Image.open(path).convert('RGB')
    img = img.resize(target_size)
    img_array = np.array(img) / 255.0
    
    # Remove white fade effect by normalizing to full range
    if img_array.max() > img_array.min():
        img_array = (img_array - img_array.min()) / (img_array.max() - img_array.min())
    
    img_tensor = torch.from_numpy(img_array).permute(2, 0, 1).float()
    return img_tensor

def load_bev_gt(path, target_size=(256, 256)):
    """Load and preprocess BEV ground truth"""
    img = Image.open(path).convert('L')
    img = img.resize(target_size)
    img_array = np.array(img)
    img_tensor = torch.from_numpy(img_array).long()
    return img_tensor

# Find all validation samples
val_data_root = "../data/cam2bev-data-master-1_FRLR/1_FRLR/val"
front_images = glob.glob(f"{val_data_root}/front/front/*.png")
print(f"📂 Found {len(front_images)} validation samples")

# Pick random samples to test
num_samples = 5  # Test 5 random samples
random_samples = random.sample(front_images, min(num_samples, len(front_images)))

print(f"🎲 Testing {len(random_samples)} random samples:")
for i, sample_path in enumerate(random_samples):
    sample_id = os.path.basename(sample_path).replace('.png', '')
    print(f"  {i+1}. {sample_id}")

# Load first random sample
sample_path = random_samples[0]
sample_id = os.path.basename(sample_path).replace('.png', '')

print(f"\n📸 Loading sample: {sample_id}")

# Load all camera views for this sample
camera_images = {}
for view in ['front', 'left', 'right', 'rear']:
    view_path = sample_path.replace('/front/front/', f'/{view}/{view}/')
    if os.path.exists(view_path):
        camera_images[view] = load_image(view_path, (config['img_height'], config['img_width']))
    else:
        print(f"⚠️  Missing {view} camera for {sample_id}")

# Load BEV ground truth
bev_path = sample_path.replace('/front/front/', '/bev/bev/')
if os.path.exists(bev_path):
    bev_gt_tensor = load_bev_gt(bev_path, (config['bev_height'], config['bev_width']))
else:
    print(f"⚠️  Missing BEV ground truth for {sample_id}")

print("✅ Sample loaded:")
for view, img in camera_images.items():
    print(f"  - {view}: {img.shape}")
print(f"  - BEV GT: {bev_gt_tensor.shape}")


📂 Found 3731 validation samples
🎲 Testing 5 random samples:
  1. v_2_0053000
  2. v_1_0089000
  3. v_3_0144500
  4. v_4_0039500
  5. v_3_0155500

📸 Loading sample: v_2_0053000
✅ Sample loaded:
  - front: torch.Size([3, 256, 256])
  - left: torch.Size([3, 256, 256])
  - right: torch.Size([3, 256, 256])
  - rear: torch.Size([3, 256, 256])
  - BEV GT: torch.Size([256, 256])


In [43]:
# --- Test Multiple Random Samples ---
# Initialize metrics calculator
class_names = {0: "unlabeled", 1: "car", 2: "vegetation", 3: "road", 4: "terrain", 5: "guard_rail", 6: "sidewalk"}
metrics_calc = MetricsCalculator(config['num_classes'], class_names)

print("🔬 Testing multiple random samples...")

all_results = []
for i, sample_path in enumerate(random_samples):
    sample_id = os.path.basename(sample_path).replace('.png', '')
    print(f"\n--- Sample {i+1}/{len(random_samples)}: {sample_id} ---")
    
    # Load camera images for this sample
    camera_images = {}
    for view in ['front', 'left', 'right', 'rear']:
        view_path = sample_path.replace('/front/front/', f'/{view}/{view}/')
        if os.path.exists(view_path):
            camera_images[view] = load_image(view_path, (config['img_height'], config['img_width']))
    
    # Load BEV ground truth
    bev_path = sample_path.replace('/front/front/', '/bev/bev/')
    if not os.path.exists(bev_path):
        print(f"⚠️  Skipping {sample_id} - missing BEV ground truth")
        continue
        
    bev_gt_tensor = load_bev_gt(bev_path, (config['bev_height'], config['bev_width']))
    
    # Run inference
    camera_inputs = {view: img.unsqueeze(0).to(device) for view, img in camera_images.items()}
    bev_gt_input = bev_gt_tensor.unsqueeze(0).to(device)
    
    with torch.no_grad():
        predictions = model(camera_inputs)
    
    # Calculate metrics
    metrics = metrics_calc.calculate_metrics(predictions, bev_gt_input)
    
    # Store results
    result = {
        'sample_id': sample_id,
        'accuracy': metrics['accuracy'],
        'miou': metrics['mean_iou'],
        'present_classes': metrics['present_classes']
    }
    all_results.append(result)
    
    print(f"  Accuracy: {metrics['accuracy']:.4f}")
    print(f"  mIoU: {metrics['mean_iou']:.4f}")
    print(f"  Classes: {metrics['present_classes']}")

# Summary statistics
if all_results:
    avg_accuracy = np.mean([r['accuracy'] for r in all_results])
    avg_miou = np.mean([r['miou'] for r in all_results])
    
    print(f"\n📊 Summary across {len(all_results)} samples:")
    print(f"  Average Accuracy: {avg_accuracy:.4f}")
    print(f"  Average mIoU: {avg_miou:.4f}")
    
    # Show best and worst samples
    best_sample = max(all_results, key=lambda x: x['miou'])
    worst_sample = min(all_results, key=lambda x: x['miou'])
    
    print(f"  Best mIoU: {best_sample['sample_id']} ({best_sample['miou']:.4f})")
    print(f"  Worst mIoU: {worst_sample['sample_id']} ({worst_sample['miou']:.4f})")


🔬 Testing multiple random samples...

--- Sample 1/5: v_2_0053000 ---
  Accuracy: 0.0000
  mIoU: 0.0000
  Classes: 6

--- Sample 2/5: v_1_0089000 ---
  Accuracy: 0.0012
  mIoU: 0.0097
  Classes: 7

--- Sample 3/5: v_3_0144500 ---
  Accuracy: 0.0000
  mIoU: 0.0000
  Classes: 0

--- Sample 4/5: v_4_0039500 ---
  Accuracy: 0.0000
  mIoU: 0.0000
  Classes: 2

--- Sample 5/5: v_3_0155500 ---
  Accuracy: 0.0000
  mIoU: 0.0000
  Classes: 2

📊 Summary across 5 samples:
  Average Accuracy: 0.0003
  Average mIoU: 0.0019
  Best mIoU: v_1_0089000 (0.0097)
  Worst mIoU: v_3_0144500 (0.0000)


In [44]:
# --- Visualize All Samples ---
if all_results:
    print(f"🖼️  Creating visualizations for all {len(all_results)} samples...")
    
    # Create visualizer
    visualizer = create_default_visualizer()
    
    for i, result in enumerate(all_results):
        sample_id = result['sample_id']
        print(f"  Visualizing sample {i+1}/{len(all_results)}: {sample_id} (mIoU: {result['miou']:.4f})")
        
        # Load camera images and BEV ground truth for this sample
        camera_images_viz = {}
        for view in ['front', 'left', 'right', 'rear']:
            view_path = f"../data/cam2bev-data-master-1_FRLR/1_FRLR/val/{view}/{view}/{sample_id}.png"
            if os.path.exists(view_path):
                camera_images_viz[view] = load_image(view_path, (config['img_height'], config['img_width']))
            else:
                print(f"    ⚠️  Missing {view} camera for {sample_id}")
        
        # Load BEV ground truth for this sample
        bev_path = f"../data/cam2bev-data-master-1_FRLR/1_FRLR/val/bev/bev/{sample_id}.png"
        if os.path.exists(bev_path):
            bev_gt_viz = load_bev_gt(bev_path, (config['bev_height'], config['bev_width']))
        else:
            print(f"    ⚠️  Missing BEV ground truth for {sample_id}")
            continue
        
        # Run inference on this sample
        camera_inputs_viz = {view: img.unsqueeze(0).to(device) for view, img in camera_images_viz.items()}
        bev_gt_input_viz = bev_gt_viz.unsqueeze(0).to(device)
        
        with torch.no_grad():
            predictions_viz = model(camera_inputs_viz)
        
        # Create visualization
        save_path = f"./sample_{sample_id}.png"
        
        visualizer.visualize_sample(
            camera_images=camera_inputs_viz,
            bev_features=None,
            segmentation_logits=predictions_viz,
            ground_truth=bev_gt_input_viz,
            sample_id=sample_id,
            save_path=save_path
        )
        
        print(f"    ✅ Saved as '{save_path}'")
    
    print(f"\n✅ All {len(all_results)} samples visualized and saved!")
    
    # Show best and worst samples
    best_sample = max(all_results, key=lambda x: x['miou'])
    worst_sample = min(all_results, key=lambda x: x['miou'])
    
    print(f"\n🏆 Best sample: {best_sample['sample_id']} (mIoU: {best_sample['miou']:.4f})")
    print(f"📉 Worst sample: {worst_sample['sample_id']} (mIoU: {worst_sample['miou']:.4f})")
    
else:
    print("⚠️  No results to visualize")


🖼️  Creating visualizations for all 5 samples...
Initializing BEV Visualizer...
  - Number of classes: 7
  - Class names: ['unlabeled', 'car', 'vegetation', 'road', 'terrain', 'guard_rail', 'sidewalk']
  Visualizing sample 1/5: v_2_0053000 (mIoU: 0.0000)
    ✅ Saved as './sample_v_2_0053000.png'
  Visualizing sample 2/5: v_1_0089000 (mIoU: 0.0097)
    ✅ Saved as './sample_v_1_0089000.png'
  Visualizing sample 3/5: v_3_0144500 (mIoU: 0.0000)
    ✅ Saved as './sample_v_3_0144500.png'
  Visualizing sample 4/5: v_4_0039500 (mIoU: 0.0000)
    ✅ Saved as './sample_v_4_0039500.png'
  Visualizing sample 5/5: v_3_0155500 (mIoU: 0.0000)
    ✅ Saved as './sample_v_3_0155500.png'

✅ All 5 samples visualized and saved!

🏆 Best sample: v_1_0089000 (mIoU: 0.0097)
📉 Worst sample: v_3_0144500 (mIoU: 0.0000)
